# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [2]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [3]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [4]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [5]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [6]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [7]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [8]:
import numpy as np
from numpy.linalg import norm

def jacobi_iteration(A, b, num_iterations):
    n = len(A)
    x = np.zeros(n)
    x_new = np.zeros(n)
    
    for _ in range(num_iterations):
        for i in range(n):
            x_new[i] = (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
        x = x_new.copy()
        
        B = np.matmul(A, x) - b
        norm_B = norm(B)
        print(f"Iteration {_+1}: Norm of B = {norm_B}")
    
    return x



In [9]:
A = np.array([[10, 2, 1],
              [1, 5, 1],
              [2, 3, 10]])
b = np.array([7, -8, 6])
num_iterations = 10

result = jacobi_iteration(A, b, num_iterations)
print("Final result:", result)

Iteration 1: Norm of B = 4.4732538492690095
Iteration 2: Norm of B = 0.6782329983125271
Iteration 3: Norm of B = 0.39077870975783685
Iteration 4: Norm of B = 0.05815874826713564
Iteration 5: Norm of B = 0.04228448415199155
Iteration 6: Norm of B = 0.009958895521090685
Iteration 7: Norm of B = 0.005483374625174072
Iteration 8: Norm of B = 0.0017266477783258476
Iteration 9: Norm of B = 0.0007873061769432243
Iteration 10: Norm of B = 0.0002823592107246779
Final result: [ 1.00001027 -1.99998396  1.00001466]


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [10]:
import numpy as np
from numpy.linalg import norm

def gauss_seidel_iteration(A, b, num_iterations):
    n = len(A)
    x = np.zeros(n)
    
    for _ in range(num_iterations):
        for i in range(n):
            x[i] = (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    
        B = np.matmul(A, x) - b
        norm_B = norm(B)
        print(f"Iteration {_+1}: Norm of B = {norm_B}")
    
    return x


In [11]:
n = 5  # Size of the matrix
A = np.random.rand(n, n)
b = np.random.rand(n)
num_iterations = 10

result = gauss_seidel_iteration(A, b, num_iterations)
print("Final result:", result)

Iteration 1: Norm of B = 202.67938024009368
Iteration 2: Norm of B = 20189.717851304937
Iteration 3: Norm of B = 1952569.1733999238
Iteration 4: Norm of B = 188604789.47848713
Iteration 5: Norm of B = 18217253146.44172
Iteration 6: Norm of B = 1759594014229.8066
Iteration 7: Norm of B = 169958167388565.2
Iteration 8: Norm of B = 1.6416160991365828e+16
Iteration 9: Norm of B = 1.585627485995254e+18
Iteration 10: Norm of B = 1.5315484087062138e+20
Final result: [-3.61900703e+18  1.04874310e+18  2.28794796e+18 -1.13800780e+20
  4.11944927e+18]


# III. Minimum residual scheme

 ### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [12]:
import numpy as np
from numpy.linalg import norm, solve


def minres(A, b, num_iterations):
    n = len(A)
    x = np.zeros(n)
    r = b - np.dot(A, x)
    p = r.copy()
    norm_b = norm(b)
    
    for k in range(num_iterations):
        Ap = np.dot(A, p)
        alpha = np.dot(r, r) / np.dot(p, Ap)
        x += alpha * p
        r -= alpha * Ap
        
        residual_norm = norm(r)
        deviation = norm(x - solve(A, b))
        tau = np.dot(r, Ap) / np.dot(Ap, Ap)
        
        print(f"Iteration {k+1}: Residual Norm = {residual_norm}, Deviation = {deviation}, Tau = {tau}")
        
        if residual_norm / norm_b < 1e-6:
            break
        
        beta = np.dot(r, r) / np.dot(Ap, Ap)
        p = r + beta * p
    
    return x


In [13]:
n = 5  # Size of the matrix
A = np.random.rand(n, n)
b = np.random.rand(n)
num_iterations = 10

result = minres(A, b, num_iterations)
print("Final result:", result)

Iteration 1: Residual Norm = 0.7869451337170505, Deviation = 3.4847854124617377, Tau = -0.20350339387417132
Iteration 2: Residual Norm = 0.6399674937213871, Deviation = 2.8870746505007157, Tau = -0.4273764881607348
Iteration 3: Residual Norm = 0.36084513067062746, Deviation = 2.6252614236409353, Tau = -0.234075781277884
Iteration 4: Residual Norm = 0.38883749239363674, Deviation = 2.633208942059029, Tau = -0.4720014191894415
Iteration 5: Residual Norm = 1.3460004577566667, Deviation = 3.5685548248093997, Tau = -2.9223482403616656
Iteration 6: Residual Norm = 0.7556228810943256, Deviation = 2.1468624044984668, Tau = -0.0884163522613723
Iteration 7: Residual Norm = 0.3211150040673144, Deviation = 2.293470627236445, Tau = -0.1354987474229064
Iteration 8: Residual Norm = 0.4633179436232935, Deviation = 2.538169855052374, Tau = -0.7789915746096282
Iteration 9: Residual Norm = 8.228684845345434, Deviation = 10.03004038206825, Tau = -14.138258065484017
Iteration 10: Residual Norm = 0.49272329